In [1]:
# ── Robust Project Setup ─────────────────────────────────────────────────
import os, sys
from pathlib import Path

def find_project_root(marker_dirs=('src','data')):
    p = Path().resolve()
    while True:
        if all((p / d).is_dir() for d in marker_dirs):
            return p
        if p.parent == p:
            raise RuntimeError(f"Could not find project root containing {marker_dirs}")
        p = p.parent

# 1) locate & cd into your Risk_Project folder
project_root = find_project_root()
os.chdir(project_root)

# 2) ensure Python can import src/ and config.py
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✔️ Project root: {project_root}")
print(f"✔️ CWD:          {os.getcwd()}")
print(f"✔️ Contents:     {os.listdir(project_root)}")


✔️ Project root: /Users/LeoWang/Desktop/CU/Risk/Risk_Project
✔️ CWD:          /Users/LeoWang/Desktop/CU/Risk/Risk_Project
✔️ Contents:     ['.DS_Store', 'pyproject.toml', 'tests', '.venv', 'data', 'notebooks', 'src']


In [2]:
# ── Load config, data, & calibration funcs ───────────────────────────────
from risk_project.config        import STOCK_FILES, TRADING_DAYS_YR
from risk_project.data_loader   import load_price_series
from risk_project.calibration   import compute_log_returns, estimate_mu_sigma, estimate_covariance_matrix

series = load_price_series(STOCK_FILES)

# quick check
mu, sigma = estimate_mu_sigma(series[next(iter(series))], trading_days_per_year=TRADING_DAYS_YR)
print(f"Example μ/σ on {next(iter(series))}: {mu:.4f}, {sigma:.4f}")

cov = estimate_covariance_matrix(series)
print("Covariance matrix:\n", cov)


Example μ/σ on AAPL: 0.1754, 0.4437
Covariance matrix:
           AAPL      AMZN
AAPL  0.196869  0.075444
AMZN  0.075444  0.303796


In [3]:
from risk_project.calibration import compute_log_returns, estimate_mu_sigma, estimate_covariance_matrix

# Compute and display log returns for AAPL
aapl_rets = compute_log_returns(series['AAPL'])
print("AAPL log returns head:\n", aapl_rets.head(), "\n")

# Estimate mu & sigma for both
for sym, ps in series.items():
    mu, sigma = estimate_mu_sigma(ps)
    print(f"{sym} → mu = {mu:.4f}, σ = {sigma:.4f}")

# Covariance matrix
cov = estimate_covariance_matrix(series)
print("\nAnnualized covariance matrix:\n", cov)


AAPL log returns head:
 Dates
1982-11-16   -0.050920
1982-11-17    0.043803
1982-11-18    0.000000
1982-11-19   -0.014389
1982-11-22   -0.059719
Name: PX_LAST, dtype: float64 

AAPL → mu = 0.1754, σ = 0.4437
AMZN → mu = 0.2807, σ = 0.5512

Annualized covariance matrix:
           AAPL      AMZN
AAPL  0.196869  0.075444
AMZN  0.075444  0.303796
